# Rethink Steps
## Second part of process, import output of Scraping into RethinkDB
### Then come up with some other things to do

## Imports and global variables

In [34]:
import rethinkdb as r
import os
import json
from pprint import pprint

wd = './tft'
TAFE_COURSES_IMPORT_JSON_FILE = os.path.join(wd, 'tafe-sa-qualifications1.json')
ASCED_INPUT_FILE = os.path.join(wd, 'raw_foes.txt')

what_interests_me = [
    {
        "topic": "The 'Sciences'", 
        "broad_foes": ["01", "05", "06"]
    },
    {
        "topic": "Making things", 
        "broad_foes" :["02", "03","04"]
    },
    {
        "topic": "The Arts and the Great Unknown",  # (because 12 could be anything)
        "broad_foes" : ["09", "10", "12"]
    }, 
    {
        "topic": "People", 
        "broad_foes" : ["07", "08", "11"]
    },
]

conn = r.connect("localhost", 28015)

## Helper Functions

In [23]:
def connect_to_rethinkdb_instance(host='localhost', port=28015):
    """Wrapper over rethink.connect(). Returns a connection."""
    try:
        conn = r.connect(host, port)
        return conn
    except:
        print("Unable to connect to rethinkdb on", host, ":", port)
        return

    
def load_json_file(file_name):
    """Loads a JSON file"""
    with open(file_name, 'r') as f:
        return json.load(f)
    
    
def load_json_file_into_array(file_name):
    """Loads a file in JSON format to a list. Returns None if file is not an array."""
    array = load_json_file(file_name)
    if isinstance(array, list):
        return array
    else:
        print('Imported file must be a list.')
    
    
def create_table_if_not_exists(conn, table_name, db_name='test'):
    """Creates a table if it doesn't exist"""
    try:
        r.db(db_name).table_create(table_name).run(conn)
    except:
        print(table_name, "exists on ", db_name)

        
def get_table_length(conn, table_name, db_name='test'):
    """Returns an integer value, number of records in a table"""
    return r.db(db_name).table(table_name).count().run(conn)


def table_is_empty(conn, table_name, db_name='test'):
    """Returns True if a table has 0 records"""
    return get_table_length(conn, table_name, db_name) == 0


def insert_data_into_table(conn, data, table_name, db_name='test'):
    """Wrapper over rethink.table(x).insert(data)"""
    return r.db(db_name).table(table_name).insert(data).run(conn)

    
def import_if_not_exists(file_name, table_name, db_name='test', host='localhost', port=28015):
    """Imports a JSON file into a table"""
    table_path = db_name + "." + table_name
    
    conn = connect_to_rethinkdb_instance(host, port)
    
    array = load_json_file_into_array(file_name)
    length = len(array)
    
    new_table = create_table_if_not_exists(conn, table_name, db_name)
    
    # Insert records into table ONLY if table is empty
    if table_is_empty(conn, table_name, db_name):
        print(length, "records inserted into", table_path)
        return insert_data_into_table(conn, array, table_name, db_name)
    else:
        print(table_path, "is not empty, import aborted.")


def drop_table(conn, table_name, db_name='test'):
    """Wrapper over r.table_drop(table_name)"""
    return r.db(db_name).table_drop(table_name).run(conn)


def truncate(conn, table_name, db_name='test'):
    """Wrapper over r.table(table_name).delete()"""
    return r.db(db_name).table(table_name).delete().run(conn)

# Tests and finally end up with the data in 'tafe_courses' table

In [27]:
print("""IMPORT IF NOT EXISTS: Should be 1329 Records or 0 if table already exists""")
print(import_if_not_exists(TAFE_COURSES_IMPORT_JSON_FILE, 'tafe_courses'))

IMPORT IF NOT EXISTS: Should be 1329 Records or 0 if table already exists
tafe_courses exists on  test
test.tafe_courses is not empty, import aborted.
None


In [28]:
print("""GET_TABLE_LENGTH: Should be 1329 Records""")
print(get_table_length(conn, 'tafe_courses'))

GET_TABLE_LENGTH: Should be 1329 Records
1329


In [29]:
print("""TRUNCATE: Should be 0 records""")
print(truncate(conn, 'tafe_courses'))

TRUNCATE: Should be 0 records
{'skipped': 0, 'inserted': 0, 'deleted': 1329, 'unchanged': 0, 'replaced': 0, 'errors': 0}


In [30]:
print("""TABLE_IS_EMPTY: Should be True""")
print(table_is_empty(conn, 'tafe_courses'))

TABLE_IS_EMPTY: Should be True
True


In [31]:
print("""DROP_TABLE""")
print(drop_table(conn, 'tafe_courses'))
print("""IMPORT_IF_NOT_EXISTS: Should be 1329 records""")
print(import_if_not_exists(TAFE_COURSES_IMPORT_JSON_FILE, 'tafe_courses'))

DROP_TABLE
{'config_changes': [{'old_val': {'durability': 'hard', 'shards': [{'primary_replica': 'Emils_MacBook_Air_local_j4s', 'nonvoting_replicas': [], 'replicas': ['Emils_MacBook_Air_local_j4s']}], 'primary_key': 'id', 'indexes': [], 'id': '3ad0321c-7d5b-4963-bcf8-9f6fbd6c207d', 'write_acks': 'majority', 'name': 'tafe_courses', 'db': 'test'}, 'new_val': None}], 'tables_dropped': 1}
IMPORT_IF_NOT_EXISTS: Should be 1329 records
1329 records inserted into test.tafe_courses
{'skipped': 0, 'inserted': 1329, 'deleted': 0, 'unchanged': 0, 'replaced': 0, 'generated_keys': ['1ab1022e-42ed-44ec-b0d9-bdd7d1834862', '71744280-1d51-484a-8bac-0909d16e0f89', '0fc44e23-3ff5-4198-9d14-f88a6683a588', '4a84c2fb-79e2-4f94-b45d-2a9ea0cc3168', 'ba5a3c49-c050-46dd-8657-fa660eca01cb', '5f1af41e-9362-4cad-b4ec-756ee68f412b', 'b4fbd613-d86c-437b-9388-0acca29bb7c7', '44be4c0d-213c-4d4d-99d8-c1c30bdc3adf', 'c6668d69-0222-4647-be2e-d244342d06c6', '97dc45e1-1034-4e5f-bca2-203673d2ac80', 'b398f51e-e180-4c47-8828-

In [32]:
print("""GET_TABLE_LENGTH: Should be 1329 records""")
print(get_table_length(conn, 'tafe_courses'))

GET_TABLE_LENGTH: Should be 1329 records
1329


# Load Narrow FOE information into rethinkdb 'narrow_foe' table

In [39]:
def get_narrow_foes_from(file_name):
    """Loads a very specific file and returns a list of narrow foes"""
    with open(file_name, 'r') as f:
        read_file = f.read()
    
    narrow_foes = []  # Home for a list of items containing all data for a record required
    broad_foes = {}
    # Split file data on lines, removing blank lines
    lines = read_file.split("\n")
    lines = [line for line in lines if line.strip() != ""]

    # Iterate over lines and append to #foes an item with keys: 
    #  code, description, level, broad and narrow
    for line in lines:
        code, description = line.split(None ,1)  # using None as 1st arg splits on whitespace, 
        item = {}                                # 2nd arg determines number of times split
        item["code"] = code
        item["description"] = description
        
        # Only keep the narrow foes
        if len(code) == 2:
            broad_foes[code] = description
        elif len(code) == 4:
            item["broad_code"] = code[:2]
            item["broad_description"] = broad_foes[code[:2]]
            
            narrow_foes.append(item)
        
    return narrow_foes

## Create and Insert narrow_foes into 'narrow_foes'

In [44]:
narrow_foes = get_narrow_foes_from(ASCED_INPUT_FILE)

create_table_if_not_exists(conn, 'narrow_foes')
if table_is_empty(conn, 'narrow_foes'):
    insert_data_into_table(conn, narrow_foes, 'narrow_foes')

narrow_foes exists on  test


In [46]:
db_foes = list(r.table('narrow_foes').run(conn))

In [63]:
print(len(db_foes))
test_narrow_foe = db_foes[0]

71


In [62]:
courses_09 = list(r.table('tafe_courses').filter(r.row['ASCED']['code'] == test_narrow_foe['code']).run(conn))

courses_09[0]['Course Name']
    

'Diploma of Legal Services'